<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/NumericalSymbolicAutomaticDerivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax.numpy as jnp
import jax
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
def f(x):
    return jnp.cos(x)

In [3]:
def get_d2f(f):

    def d2f(x,h):
        return (f(x+h)-2*f(x)+f(x-h))/h**2

    return d2f

In [4]:
d2f=get_d2f(f)
jax_d2f=jax.grad(jax.grad(f))

In [5]:
for h in range(0,-16,-1):
    
    print(f'{h:3d}, {d2f(np.pi/6,10**h):25.16e} {-np.cos(np.pi/6):25.16e}  {jax_d2f(np.pi/6):25.16e}')

  0,   -7.9621976235863934e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -1,   -8.6530395646761105e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -2,   -8.6601818693132770e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -3,   -8.6602533166946216e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -4,   -8.6602541804481348e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -5,   -8.6602613968977937e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -6,   -8.6608498151008462e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -7,   -8.7707618945387378e-01   -8.6602540378443871e-01    -8.6602540378443871e-01
 -8,   -1.1102230246251563e+00   -8.6602540378443871e-01    -8.6602540378443871e-01
 -9,   -1.1102230246251564e+02   -8.6602540378443871e-01    -8.6602540378443871e-01
-10,    0.0000000000000000e+00   -8.6602540378443871e-01    -8.6602540378443871e-01
-11,    0.0000000000000000e+00   -8.6602540378443871e-01    -8.6602540378443

In [6]:
jax_d2f(np.pi/6)

DeviceArray(-0.8660254, dtype=float64, weak_type=True)